# How to Grid Search Hyperparameters for Deep Learning Models in Python With Keras
Hyperparameter optimization is a big part of deep learning.

The reason is that neural networks are notoriously difficult to configure and there are a lot of parameters that need to be set. On top of that, individual models can be very slow to train.

In this post you will discover how you can use the grid search capability from the scikit-learn python machine learning library to tune the hyperparameters of Keras deep learning models.

After reading this post you will know:

* How to wrap Keras models for use in scikit-learn and how to use grid search.
* How to grid search common neural network parameters such as learning rate, dropout rate, epochs and number of neurons.
* How to define your own hyperparameter tuning experiments on your own projects.

Let’s get started.

## Overview
In this post, I want to show you both how you can use the scikit-learn grid search capability and give you a suite of examples that you can copy-and-paste into your own project as a starting point.

Below is a list of the topics we are going to cover:

1. How to use Keras models in scikit-learn.
2. How to use grid search in scikit-learn.
3. How to tune batch size and training epochs.
4. How to tune optimization algorithms.
5. How to tune learning rate and momentum.
6. How to tune network weight initialization.
7. How to tune activation functions.
8. How to tune dropout regularization.
9. How to tune the number of neurons in the hidden layer.

## How to Use Keras Models in scikit-learn
Keras models can be used in scikit-learn by wrapping them with the KerasClassifier or KerasRegressor class.

To use these wrappers you must define a function that creates and returns your Keras sequential model, then pass this function to the build_fn argument when constructing the KerasClassifier class.

For example:

In [ ]:
def create_model():
	...
	return model
 
model = KerasClassifier(build_fn=create_model)

The constructor for the KerasClassifier class can take default arguments that are passed on to the calls to model.fit(), such as the number of epochs and the batch size.

For example:

In [ ]:
def create_model():
	...
	return model
 
model = KerasClassifier(build_fn=create_model, epochs=10)

The constructor for the KerasClassifier class can also take new arguments that can be passed to your custom create_model() function. These new arguments must also be defined in the signature of your create_model() function with default parameters.

For example:

In [ ]:
def create_model(dropout_rate=0.0):
	...
	return model
 
model = KerasClassifier(build_fn=create_model, dropout_rate=0.2)

## How to Use Grid Search in scikit-learn
Grid search is a model hyperparameter optimization technique.

In scikit-learn this technique is provided in the GridSearchCV class.

When constructing this class you must provide a dictionary of hyperparameters to evaluate in the param_grid argument. This is a map of the model parameter name and an array of values to try.

By default, accuracy is the score that is optimized, but other scores can be specified in the score argument of the GridSearchCV constructor.

By default, the grid search will only use one thread. By setting the n_jobs argument in the GridSearchCV constructor to -1, the process will use all cores on your machine. Depending on your Keras backend, this may interfere with the main neural network training process.

The GridSearchCV process will then construct and evaluate one model for each combination of parameters. Cross validation is used to evaluate each individual model and the default of 3-fold cross validation is used, although this can be overridden by specifying the cv argument to the GridSearchCV constructor.

Below is an example of defining a simple grid search:

In [ ]:
param_grid = dict(epochs=[10,20,30])
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)

Once completed, you can access the outcome of the grid search in the result object returned from grid.fit(). The best_score_ member provides access to the best score observed during the optimization procedure and the best_params_ describes the combination of parameters that achieved the best results.

## Problem Description
Now that we know how to use Keras models with scikit-learn and how to use grid search in scikit-learn, let’s look at a bunch of examples.

All examples will be demonstrated on a small standard machine learning dataset called the [Pima Indians onset of diabetes classification dataset](http://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes). This is a small dataset with all numerical attributes that is easy to work with.

1. [Download the dataset](http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data) and place it in your currently working directly with the name pima-indians-diabetes.csv.

As we proceed through the examples in this post, we will aggregate the best parameters. This is not the best way to grid search because parameters can interact, but it is good for demonstration purposes.

## How to Tune Batch Size and Number of Epochs
In this first simple example, we look at tuning the batch size and number of epochs used when fitting the network.

The batch size in [iterative gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method) is the number of patterns shown to the network before the weights are updated. It is also an optimization in the training of the network, defining how many patterns to read at a time and keep in memory.

The number of epochs is the number of times that the entire training dataset is shown to the network during training. Some networks are sensitive to the batch size, such as LSTM recurrent neural networks and Convolutional Neural Networks.

Here we will evaluate a suite of different mini batch sizes from 10 to 100 in steps of 20.

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.696615 using {'epochs': 100, 'batch_size': 20}
0.585938 (0.030425) with: {'epochs': 10, 'batch_size': 10}
0.447917 (0.164011) with: {'epochs': 50, 'batch_size': 10}
0.468750 (0.164712) with: {'epochs': 100, 'batch_size': 10}
0.492188 (0.123031) with: {'epochs': 10, 'batch_size': 20}
0.544271 (0.155804) with: {'epochs': 50, 'batch_size': 20}
0.696615 (0.015073) with: {'epochs': 100, 'batch_size': 20}
0.553385 (0.077886) with: {'epochs': 10, 'batch_size': 40}
0.522135 (0.128939) with: {'epochs': 50, 'batch_size': 40}
0.665365 (0.010253) with: {'epochs': 100, 'batch_size': 40}
0.520833 (0.083435) with: {'epochs': 10, 'batch_size': 60}
0.526042 (0.143430) with: {'epochs': 50, 'batch_size': 60}
0.669271 (0.006639) with: {'epochs': 100, 'batch_size': 60}
0.626302 (0.039365) with: {'epochs': 10, 'batch_size': 80}
0.645833 (0.032578) with: {'epochs': 50, 'batch_size': 80}
0.571615 (0.176354) with: {'epochs': 100, 'batch_size': 80}
0.520833 (0.144982) with: {'epochs': 10, 'batch_size': 

We can see that the batch size of 20 and 100 epochs achieved the best result of about 69.6% accuracy.

## How to Tune the Training Optimization Algorithm
Keras offers a suite of different state-of-the-art optimization algorithms.

In this example, we tune the optimization algorithm used to train the network, each with default parameters.

This is an odd example, because often you will choose one approach a priori and instead focus on tuning its parameters on your problem (e.g. see the next example).

Here we will evaluate the [suite of optimization algorithms supported by the Keras API](https://keras.io/optimizers/).

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.691406 using {'optimizer': 'Adamax'}
0.651042 (0.024774) with: {'optimizer': 'SGD'}
0.666667 (0.016367) with: {'optimizer': 'RMSprop'}
0.661458 (0.018136) with: {'optimizer': 'Adagrad'}
0.602865 (0.093442) with: {'optimizer': 'Adadelta'}
0.686198 (0.003683) with: {'optimizer': 'Adam'}
0.691406 (0.009568) with: {'optimizer': 'Adamax'}
0.686198 (0.012075) with: {'optimizer': 'Nadam'}


The results suggest that the Adamax optimization algorithm is the best with a score of about 69.1% accuracy.

## How to Tune Learning Rate and Momentum
It is common to pre-select an optimization algorithm to train your network and tune its parameters.

By far the most common optimization algorithm is plain old [Stochastic Gradient Descent (SGD)](https://keras.io/optimizers/#sgd) because it is so well understood. In this example, we will look at optimizing the SGD learning rate and momentum parameters.

Learning rate controls how much to update the weight at the end of each batch and the momentum controls how much to let the previous update influence the current weight update.

We will try a suite of small standard learning rates and a momentum values from 0.2 to 0.8 in steps of 0.2, as well as 0.9 (because it can be a popular value in practice).

Generally, it is a good idea to also include the number of epochs in an optimization like this as there is a dependency between the amount of learning per batch (learning rate), the number of updates per epoch (batch size) and the number of epochs.

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.690104 using {'learn_rate': 0.001, 'momentum': 0.8}
0.572917 (0.134575) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.533854 (0.149269) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.647135 (0.021236) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.677083 (0.024360) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.690104 (0.026748) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.533854 (0.149269) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.533854 (0.149269) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.533854 (0.149269) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.348958 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.572917 (0.134575) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.533854 (0.149269) with: {'learn_rate':

We can see that relatively SGD is not very good on this problem, nevertheless best results were achieved using a learning rate of 0.001 and a momentum of 0.8 with an accuracy of about 69%.

## How to Tune Network Weight Initialization
Neural network weight initialization used to be simple: use small random values.

Now there is a suite of different techniques to choose from. [Keras provides a laundry list](https://keras.io/initializers/).

In this example, we will look at tuning the selection of network weight initialization by evaluating all of the available techniques.

We will use the same weight initialization method on each layer. Ideally, it may be better to use different weight initialization schemes according to the activation function used on each layer. In the example below we use rectifier for the hidden layer. We use sigmoid for the output layer because the predictions are binary.

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the weight initialization
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(init_mode='uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init_mode, activation='relu'))
	model.add(Dense(1, kernel_initializer=init_mode, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.723958 using {'init_mode': 'normal'}
0.709635 (0.013279) with: {'init_mode': 'uniform'}
0.692708 (0.015073) with: {'init_mode': 'lecun_uniform'}
0.723958 (0.017566) with: {'init_mode': 'normal'}
0.651042 (0.024774) with: {'init_mode': 'zero'}
0.713542 (0.010253) with: {'init_mode': 'glorot_normal'}
0.701823 (0.012075) with: {'init_mode': 'glorot_uniform'}
0.466146 (0.149269) with: {'init_mode': 'he_normal'}
0.660156 (0.012758) with: {'init_mode': 'he_uniform'}


We can see that the best results were achieved with a normal weight initialization scheme achieving a performance of about 72.4%.

## How to Tune the Neuron Activation Function
The activation function controls the non-linearity of individual neurons and when to fire.

Generally, the rectifier activation function is the most popular, but it used to be the sigmoid and the tanh functions and these functions may still be more suitable for different problems.

In this example, we will evaluate the suite of [different activation functions available in Keras](https://keras.io/activations/). We will only use these functions in the hidden layer, as we require a sigmoid activation function in the output for the binary classification problem.

Generally, it is a good idea to prepare data to the range of the different transfer functions, which we will not do in this case.

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the activation function
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation=activation))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.718750 using {'activation': 'relu'}
0.661458 (0.011201) with: {'activation': 'softmax'}
0.713542 (0.024360) with: {'activation': 'softplus'}
0.678385 (0.037783) with: {'activation': 'softsign'}
0.718750 (0.022999) with: {'activation': 'relu'}
0.661458 (0.018688) with: {'activation': 'tanh'}
0.691406 (0.016877) with: {'activation': 'sigmoid'}
0.678385 (0.015073) with: {'activation': 'hard_sigmoid'}
0.705729 (0.001841) with: {'activation': 'linear'}


Surprisingly (to me at least), the ‘relu’ activation function achieved the best results with an accuracy of about 71.9%.

## How to Tune Dropout Regularization
In this example, we will look at tuning the dropout rate for regularization in an effort to limit overfitting and improve the model’s ability to generalize.

To get good results, dropout is best combined with a weight constraint such as the max norm constraint.

This involves fitting both the dropout percentage and the weight constraint. We will try dropout percentages between 0.0 and 0.9 (1.0 does not make sense) and maxnorm weight constraint values between 0 and 5.

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the dropout rate
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(weight_constraint)))
	model.add(Dropout(dropout_rate))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.726563 using {'dropout_rate': 0.3, 'weight_constraint': 3}
0.717448 (0.009744) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.707031 (0.006379) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.712240 (0.025780) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.717448 (0.020505) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.722656 (0.024080) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.720052 (0.026748) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.707031 (0.008438) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.718750 (0.016877) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.717448 (0.016053) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.713542 (0.006639) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.700521 (0.020505) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.722656 (0.025315) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.718750 (0.019401) with: {'dropout_rate': 0.2, 'weight_constraint': 

We can see that the dropout rate of 0.3% and the maxnorm weight constraint of 3 resulted in the best accuracy of about 72.6%.

## How to Tune the Number of Neurons in the Hidden Layer
The number of neurons in a layer is an important parameter to tune. Generally the number of neurons in a layer controls the representational capacity of the network, at least at that point in the topology.

Also, generally, a large enough single layer network can approximate any other neural network, [at least in theory](https://en.wikipedia.org/wiki/Universal_approximation_theorem).

In this example, we will look at tuning the number of neurons in a single hidden layer. We will try values from 1 to 30 in steps of 5.

A larger network requires more training and at least the batch size and number of epochs should ideally be optimized with the number of neurons.

The full code listing is provided below.

In [1]:
# Use scikit-learn to grid search the number of neurons
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
	# create model
	model = Sequential()
	model.add(Dense(neurons, input_dim=8, kernel_initializer='uniform', activation='linear', kernel_constraint=maxnorm(4)))
	model.add(Dropout(0.2))
	model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Using TensorFlow backend.


Best: 0.722656 using {'neurons': 30}
0.701823 (0.012890) with: {'neurons': 1}
0.712240 (0.009744) with: {'neurons': 5}
0.707031 (0.025315) with: {'neurons': 10}
0.710938 (0.022326) with: {'neurons': 15}
0.717448 (0.025780) with: {'neurons': 20}
0.722656 (0.022097) with: {'neurons': 25}
0.722656 (0.019401) with: {'neurons': 30}


We can see that the best results were achieved with a network with 5 neurons in the hidden layer with an accuracy of about 72%.

## Tips for Hyperparameter Optimization
This section lists some handy tips to consider when tuning hyperparameters of your neural network.

* k-fold Cross Validation. You can see that the results from the examples in this post show some variance. A default cross-validation of 3 was used, but perhaps k=5 or k=10 would be more stable. Carefully choose your cross validation configuration to ensure your results are stable.
* Review the Whole Grid. Do not just focus on the best result, review the whole grid of results and look for trends to support configuration decisions.
* Parallelize. Use all your cores if you can, neural networks are slow to train and we often want to try a lot of different parameters. Consider spinning up a lot of AWS instances.
* Use a Sample of Your Dataset. Because networks are slow to train, try training them on a smaller sample of your training dataset, just to get an idea of general directions of parameters rather than optimal configurations.
* Start with Coarse Grids. Start with coarse-grained grids and zoom into finer grained grids once you can narrow the scope.
* Do not Transfer Results. Results are generally problem specific. Try to avoid favorite configurations on each new problem that you see. It is unlikely that optimal results you discover on one problem will transfer to your next project. Instead look for broader trends like number of layers or relationships between parameters.
* Reproducibility is a Problem. Although we set the seed for the random number generator in NumPy, the results are not 100% reproducible. There is more to reproducibility when grid searching wrapped Keras models than is presented in this post.

## Summary
In this post, you discovered how you can tune the hyperparameters of your deep learning networks in Python using Keras and scikit-learn.

Specifically, you learned:

* How to wrap Keras models for use in scikit-learn and how to use grid search.
* How to grid search a suite of different standard neural network parameters for Keras models.
* How to design your own hyperparameter optimization experiments.